In [204]:
import pandas as pd
import re
import numpy as np

In [205]:
data = {
    "Date": ["12.16","12.17", "12.18", "12.19", "12.20", "12.21", "12.22"], #12.24
    "Fifi": [0, 0, 0, -1.2, -1.1, 0.8, -0.9],
    "Sofya": [0, -0.6, -1.8, 0, -1, -0.4, 0.6],
    "Nick": [0, -0.6, -0.6, -0.3, -1, 0, 0],
    "Blue": [0, -1.2, -0.6, -0.6, -1, -0.2, 0.4],
    "Joyce": [0, -0.1, 0, -1.2, -0.7, 1, -1.4],
    "Jenny": [0, -1.9, -0.2, -0.4, 1, -1, 0.2],
    "Yoyo": [0, -0.8, -0.4, -0.6, 0.2, -0.8, 0.4],
    "Jimmy": [0, 0, 0, 0.3, -0.6, -0.6, -0.8],
    "Jerry": [0, -0.7, -0.3, -1.9, -0.1, 1.3, 0.6],
    "Summer G": [0, -1.5, -2.4, -0.4, 0.1, -0.8, -0.4],
    "Barry": [0, 0, 0, 0, 0.8, 1.4, -2.2],
    "Esther": [0, -0.2, 0.2, -0.7, -1, -1, 0],
    "Ginger": [0, -0.4, -1.4, -0.8, 0, -1.8, 0.4],
    "Mia": [0, -0.6, -0.8, -0.4, -0.8, -0.6, -0.6]
}

df = pd.DataFrame(data)


In [206]:
df

,Date,Fifi,Sofya,Nick,Blue,Joyce,Jenny,Yoyo,Jimmy,Jerry,Summer G,Barry,Esther,Ginger,Mia
0,12.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.17,0.0,-0.6,-0.6,-1.2,-0.1,-1.9,-0.8,0.0,-0.7,-1.5,0.0,-0.2,-0.4,-0.6
2,12.18,0.0,-1.8,-0.6,-0.6,0.0,-0.2,-0.4,0.0,-0.3,-2.4,0.0,0.2,-1.4,-0.8
3,12.19,-1.2,0.0,-0.3,-0.6,-1.2,-0.4,-0.6,0.3,-1.9,-0.4,0.0,-0.7,-0.8,-0.4
4,12.20,-1.1,-1.0,-1.0,-1.0,-0.7,1.0,0.2,-0.6,-0.1,0.1,0.8,-1.0,0.0,-0.8
5,12.21,0.8,-0.4,0.0,-0.2,1.0,-1.0,-0.8,-0.6,1.3,-0.8,1.4,-1.0,-1.8,-0.6
6,12.22,-0.9,0.6,0.0,0.4,-1.4,0.2,0.4,-0.8,0.6,-0.4,-2.2,0.0,0.4,-0.6


In [207]:

csv_file_path = '../data/trial.csv'
df.to_csv(csv_file_path, index=False)

print(f'Data has been written to {csv_file_path}')


Data has been written to ../data/trial.csv


In [208]:
goals = {
    "Fifi": -3,
    "Sofya": -4,
    "Nick": -4,
    "Blue": -5,
    "Joyce": -5,
    "Jenny": -5,
    "Joker": -5,
    "Jimmy": -5,
    "Jerry": -10,
    "Summer G": -5,
    "Barry": -8,
    "Esther": -10,
    "Ginger": -10,
    "Mia": -10
}


In [209]:
import re

def parse_date_and_names(message):
    """
    Extracts the date and parses names and scores from a given message.

    Parameters:
        message (str): The string message containing the date and name-score pairs.

    Returns:
        tuple:
            date (str): Extracted date in 'MM.DD' format or 'Date Not Found' if no valid date is found.
            parsed_data (dict): A dictionary where keys are standard names and values are scores (float).
    """
    lines = message.split('\n')
    
    # Normalize non-standard characters for + and -
    message = message.replace('－', '-').replace('➕', '+').replace('＋', '+').replace('–', '-')

    # Extract date pattern
    date_pattern = r'\d{2}\.\d{2}(-D\d+|共|Day\d+)?'
    date = "Date Not Found"
    for line in lines:
        date_match = re.search(date_pattern, line)
        if date_match:
            date = date_match.group(0).split('-')[0]  # Extract only the 'MM.DD' part
            break

    # Mapping for name replacements
    name_mapping = {
        "💄ʚ 脚丫 ɞ 🐾": "Jenny",
        "叶卡卡": "Blue",
        "Summer樂": "Summer G",
        "joker": "Yoyo",
        "jerry": "Jerry",
        "Jimmy": "Jimmy",
        "Esther Gu": "Esther",
        "Barry": "Barry",
        "Joyce💗": "Joyce",
        "Mia.L": "Mia",
        "Sofya™": "Sofya",
        "Ginger姜睿辰": "Ginger",
        "瞿俊彦": "Nick",
        "fifi": "Fifi"
    }

    # Dictionary to hold parsed data
    parsed_data = {}
    # Pattern to extract scores
    score_pattern = r"([-+]?\d*\.?\d+)"

    # Iterate through lines to find names and scores
    for line in lines[2:]:  # Assuming the first two lines are not data lines
        # Split line into parts
        parts = line.split(' ', 2)
        if len(parts) < 3:
            continue
        
        # Find the score in the line
        score_matches = re.findall(score_pattern, parts[2])
        if score_matches:
            score = float(score_matches[0])
            # Check if there's a weird plus sign or no sign, assume negative if not specified
            if '＋' not in line and '➕' not in line and not score_matches[0].startswith(('+', '-')):
                score = -abs(score)
        else:
            score = 0.0

        # Match names using the name mapping
        for key, value in name_mapping.items():
            if key in line:
                parsed_data[value] = score
                break

    return date, parsed_data


In [201]:
message24 = """
#接龙
12.24-D9

1. jerry -1.3，共-3.7
2. Jimmy +0.1
3. Esther Gu –1.0
4. 💄ʚ 脚丫 ɞ 🐾 ᥫᩣ -0.5，共2.6
5. Barry -1.5
6. Joyce💗 0
7. Mia.L 0，共-3.8
8. 叶卡卡。 +0.8
9. Summer樂小樂⁶⁶ 👼🏻 -0.6共-7.1
10. Sofya™ +0.8
11. joker 0
12. Ginger姜睿辰 -2.0 共 -5.2
"""
message23 ="""
#接龙
12.23-D8

1. jerry -1.3，共-2.4
2. Jimmy -0.1
3. Esther Gu +1.5
4. Sofya™ +1.2
5. Mia.L 0，共-3.8
6. joker -0.4 共-2.4
7. 叶卡卡。 -0.2
8. Summer樂小樂⁶⁶ 👼🏻 -1.1共-6.5
9. Joyce💗 ＋0.9
10. 翁子昊 Barry +0.8
11. 💄ʚ 脚丫 ɞ 🐾 ᥫᩣ ➕0.2，共2.1
12. Ginger姜睿辰 +0.8 共 -3.2
"""
message25 = """
#接龙
Day10-12.25

1. 💄ʚ 脚丫 ɞ 🐾 ᥫᩣ -0.4，共3
2. Esther Gu －1
3. Jimmy +1.3
4. jerry +1.9，共-1.8
5. Ginger姜睿辰 +1.2共-4
6. Summer樂小樂⁶⁶ 👼🏻 -0.1共-7.2
7. 叶卡卡。 0
8. Joyce💗 +0.2
9. Sofya™ 0

"""

message26 = """
#接龙
Day11,-12.26

1. 💄ʚ 脚丫 ɞ 🐾 ᥫᩣ -0.4,共3.4
2. Jimmy -1.1
3. Mia.L -1.2，共5斤
4. jerry +0.3，共-1.5
5. Joyce💗 -0.4
6. 叶卡卡。 +1
7. Sofya™ -0.6
8. Ginger姜睿辰 +1.4 共 -2.6
9. Summer樂小樂⁶⁶ 👼🏻 +1.1共-6.1
"""

In [212]:
for message in [message23, message24, message25]:
    date, parsed_data = parse_date_and_names(message)
    append_to_csv(parsed_data, date, file_path=csv_file_path)
# date, parsed_data

Data successfully written to ../data/trial.csv.
Data successfully written to ../data/trial.csv.
Data successfully written to ../data/trial.csv.


/tmp/ipykernel_3676/1608352336.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_3676/1608352336.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_3676/1608352336.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [176]:
len(parsed_data)

12

In [210]:
import pandas as pd

def append_to_csv(data, date, file_path):
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"File {file_path} not found. Please check the file path.")
        return

    if date in df['Date'].values:
        idx = df[df['Date'] == date].index[0]
        for name in df.columns:
            if name != 'Date':
                df.at[idx, name] = 0
        for name, score in data.items():
            df.at[idx, name] = score
    else:
        new_row = {'Date': date, **{name: 0 for name in df.columns if name != 'Date'}}
        new_row.update(data)
        df = df.append(new_row, ignore_index=True)

    df.to_csv(file_path, index=False)
    print(f"Data successfully written to {file_path}.")


In [178]:

append_to_csv(parsed_data, date, file_path=csv_file_path)

Data successfully written to ../data/trial.csv.


/tmp/ipykernel_3676/1608352336.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [214]:
csv_file_path = '../data/trial.csv'
df= pd.read_csv(csv_file_path)
df

,Date,Fifi,Sofya,Nick,Blue,Joyce,Jenny,Yoyo,Jimmy,Jerry,Summer G,Barry,Esther,Ginger,Mia
0,12.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.17,0.0,-0.6,-0.6,-1.2,-0.1,-1.9,-0.8,0.0,-0.7,-1.5,0.0,-0.2,-0.4,-0.6
2,12.18,0.0,-1.8,-0.6,-0.6,0.0,-0.2,-0.4,0.0,-0.3,-2.4,0.0,0.2,-1.4,-0.8
3,12.19,-1.2,0.0,-0.3,-0.6,-1.2,-0.4,-0.6,0.3,-1.9,-0.4,0.0,-0.7,-0.8,-0.4
4,12.20,-1.1,-1.0,-1.0,-1.0,-0.7,1.0,0.2,-0.6,-0.1,0.1,0.8,-1.0,0.0,-0.8
5,12.21,0.8,-0.4,0.0,-0.2,1.0,-1.0,-0.8,-0.6,1.3,-0.8,1.4,-1.0,-1.8,-0.6
6,12.22,-0.9,0.6,0.0,0.4,-1.4,0.2,0.4,-0.8,0.6,-0.4,-2.2,0.0,0.4,-0.6
7,12.23,0.0,1.2,0.0,-0.2,0.9,0.2,-0.4,-0.1,-1.3,-1.1,0.8,1.5,0.8,-0.0
8,12.24,0.0,0.8,0.0,0.8,-0.0,-0.5,-0.0,0.1,-1.3,-0.6,-1.5,-1.0,-2.0,-0.0
9,12.25,0.0,-0.0,0.0,-0.0,0.2,-0.4,0.0,1.3,1.9,-0.1,0.0,-1.0,1.2,0.0


In [82]:
data = {
    "Date": ["12.17", "12.18", "12.19", "12.20", "12.21", "12.22"], #12.24
    "Fifi": [0, 0, -1.2, -1.1, 0.8, -0.9],
    "Sofya": [-0.6, -1.8, 0, -1, -0.4, 0.6],
    "Nick": [-0.6, -0.6, -0.3, -1, 0, 0],
    "Blue": [-1.2, -0.6, -0.6, -1, -0.2, 0.4],
    "Joyce": [-0.1, 0, -1.2, -0.7, 1, -1.4],
    "Jenny": [-1.9, -0.2, -0.4, 1, -1, 0.2],
    "Yoyo": [-0.8, -0.4, -0.6, 0.2, -0.8, 0.4],
    "Jimmy": [0, 0, 0.3, -0.6, -0.6, -0.8],
    "Jerry": [-0.7, -0.3, -1.9, -0.1, 1.3, 0.6],
    "Summer G": [-1.5, -2.4, -0.4, 0.1, -0.8, -0.4],
    "Barry": [0, 0, 0, 0.8, 1.4, -2.2],
    "Esther": [-0.2, 0.2, -0.7, -1, -1, 0],
    "Ginger": [-0.4, -1.4, -0.8, 0, -1.8, 0.4],
    "Mia": [-0.6, -0.8, -0.4, -0.8, -0.6, -0.6]
}
name_mapping = {
        "💄ʚ 脚丫 ɞ 🐾": "Jenny",
        "叶卡卡": "Blue",
        "Summer樂": "Summer G",
        "joker": "Yoyo",
        "jerry": "Jerry",
        "Jimmy": "Jimmy",
        "Esther Gu": "Esther",
        "Barry": "Barry",
        "Joyce💗": "Joyce",
        "Mia.L": "Mia",
        "Sofya™": "Sofya",
        "Ginger姜睿辰": "Ginger",
        "瞿俊彦": "Nick"
    }

In [85]:
[i for i in data.keys() if i not in name_mapping.values()]

['Date', 'Fifi']

In [123]:
df

,Date,Fifi,Sofya,Nick,Blue,Joyce,Jenny,Yoyo,Jimmy,Jerry,Summer G,Barry,Esther,Ginger,Mia
0,12.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.17,0.0,-0.6,-0.6,-1.2,-0.1,-1.9,-0.8,0.0,-0.7,-1.5,0.0,-0.2,-0.4,-0.6
2,12.18,0.0,-1.8,-0.6,-0.6,0.0,-0.2,-0.4,0.0,-0.3,-2.4,0.0,0.2,-1.4,-0.8
3,12.19,-1.2,0.0,-0.3,-0.6,-1.2,-0.4,-0.6,0.3,-1.9,-0.4,0.0,-0.7,-0.8,-0.4
4,12.20,-1.1,-1.0,-1.0,-1.0,-0.7,1.0,0.2,-0.6,-0.1,0.1,0.8,-1.0,0.0,-0.8
5,12.21,0.8,-0.4,0.0,-0.2,1.0,-1.0,-0.8,-0.6,1.3,-0.8,1.4,-1.0,-1.8,-0.6
6,12.22,-0.9,0.6,0.0,0.4,-1.4,0.2,0.4,-0.8,0.6,-0.4,-2.2,0.0,0.4,-0.6


In [125]:
df.Ginger

0    0.0
1   -0.4
2   -1.4
3   -0.8
4    0.0
5   -1.8
6    0.4
7    0.8
8   -2.0
Name: Ginger, dtype: float64

### Cum

In [127]:
cumulative_df = df.copy()
cumulative_df.iloc[:, 1:] = df.iloc[:, 1:].cumsum(axis=0)

In [167]:

weights = np.array([0.4, 0.3, 0.2, 0.1, 0.0])
values = list(df.Ginger.values)
for i in range(5):
    
    last_five = np.array(values[-5:])
    proj = round(last_five@weights,2)
    values.append(proj)
    print(proj)

cum_projections = [np.round(cumulative_df.Ginger.values[-1],2)]
for idx,val in enumerate(values[-5:]):
    proj=np.round(cum_projections[idx]+val,2)
    cum_projections.append(proj)
    

-0.38
-0.64
-0.04
-0.42
-1.05


In [168]:
cum_projections

[-5.2, -5.58, -6.22, -6.26, -6.68, -7.73]

In [169]:
weights = np.array([0.4, 0.3, 0.2, 0.1, 0.0])

# Start with the actual values for Ginger
values = list(df.Ginger.values)

# Generate 5 projected differences
for i in range(5):
    if len(values) < 5:
        raise ValueError("Not enough values to compute projections.")
    last_five = np.array(values[-5:])  # Get the last 5 values
    proj = round(last_five @ weights, 2)  # Compute weighted average
    values.append(proj)  # Append the projection
    print(f"Projection {i+1}: {proj}")

# Compute cumulative projections
cum_projections = [np.round(cumulative_df.Ginger.values[-1], 2)]  # Start with last cumulative value
for idx, val in enumerate(values[-5:]):  # Use the last 5 projections
    proj = np.round(cum_projections[idx] + val, 2)  # Add to cumulative sum
    cum_projections.append(proj)

# Print results for debugging
print("Final Cumulative Projections:", cum_projections)


Projection 1: -0.38
Projection 2: -0.64
Projection 3: -0.04
Projection 4: -0.42
Projection 5: -1.05
Final Cumulative Projections: [-5.2, -5.58, -6.22, -6.26, -6.68, -7.73]


In [215]:
import json

In [238]:
values = list(df.Ginger.values[:-1])

In [244]:
# Initialize the list of cumulative projections with the last cumulative value
values = list(df.Ginger.values[:-1])
cumulative_projections = [-2.5]
weights = [0.08, 0.15, 0.10, 0.17, 0.12, 0.18, 0.20]
motivation_factor = 1.1
randomness_scale = 0.5

# Generate future projections
for _ in range(5):
    last_seven = np.array(values[-7:])  # Take the last 7 values
    proj_diff = round(last_seven @ np.array(weights), 2)  # Weighted average for the projection
    random_adjustment = 1 + np.random.normal(0, randomness_scale)  # Generate a random factor around 1
    proj_diff *= motivation_factor * random_adjustment  # Apply the combined random and motivation factor
    values.append(proj_diff)  # Add the projection to the raw values
    new_cumulative = np.round(cumulative_projections[-1] + proj_diff, 2)  # Update cumulative projection
    cumulative_projections.append(new_cumulative)

cumulative_projections[1:]  # Skip the initial value (starting point)

[-2.73, -2.83, -2.98, -2.97, -3.17]

In [266]:
abs(np.random.normal(0, 0.1) )

0.12663530333362955